<a href="https://colab.research.google.com/github/MOOwuttichai/BSC_DPDM2023/blob/main/%E0%B8%A7%E0%B8%B4%E0%B9%80%E0%B8%84%E0%B8%A3%E0%B8%B2%E0%B8%B0%E0%B8%AB%E0%B9%8C%E0%B8%82%E0%B9%89%E0%B8%AD%E0%B8%A1%E0%B8%B9%E0%B8%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pythainlp
!pip install -U sentence-transformers

In [ ]:
from pythainlp.corpus import thai_stopwords
from pythainlp.tokenize import Tokenizer
from pythainlp.util import normalize
from pythainlp.corpus.common import thai_words
from sklearn.cluster import KMeans
import pandas as pd

In [ ]:
kok = pd.read_csv('/content/drive/MyDrive/เชี่ยงใหม่/Data_Docter_ศ.นพ.ชวลิต(หลังสกัด)_ใหม่!!.csv',encoding='utf-8')

In [ ]:
kok

In [ ]:
len(kok[kok['defind_cancer_with_nlp']!='ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น'])

In [ ]:
a=kok[kok['defind_cancer_with_nlp']!='ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น']

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/เชี่ยงใหม่/Data_Docter_ศ.นพ.ชวลิต(หลังสกัด)_ใหม่!!.csv',encoding='utf-8')

In [ ]:
data

In [ ]:
comment=data.groupby('name').sum().reset_index()
# สร้าง set ข้อมูลภาษาไทย
words = set(thai_words())
words.remove("โรคมะเร็ง")
name =['กระเพาะปัสสวะ','กระเพาะปัสสาวะ','เยื่อบุโพรงมดลูก','ปากมดลูก','เม็ดเลือดขาว','กระเพาะอาหาร','กระเพราะอาหาร','ต่อมไทรอยด์','ต่อมไทยรอยด์','ท่อน้ำดี']
for i in name:
    words.add(i)
def remove_stopthai(tokens):
    final = [word.lower()
            for word in tokens if word not in thai_stopwords()]
    return final
def remove_bark(tokens):
    final = [word.lower() for word in tokens if word not in [' ','(',')','@','#','-','...','.','=','+','..','1','2','3','4','5','6','7','8','9','0']]
    return final
# สร้าง list เก็บตัว nlp เพิ่อนำไปวิเคราะห์โรค อาการ เเละเพศ
list_token =[]
count = []
for i in range(len(comment)):#len(comment)
    text= comment['comments'][i]
    custom_tokenizer = Tokenizer(words)
    Token = custom_tokenizer.word_tokenize(normalize(str(text)))
    Token.append('end')
    Token_final = remove_bark(remove_stopthai(Token))
    list_token.append(Token_final)
    count.append(len(Token_final))
comment['จำนวนคำ'] = count
comment['token'] = list_token

In [ ]:
set_cancer=list(set(comment['defind_cancer_with_nlp']))

In [ ]:
set_cancer

In [ ]:
from nltk.probability import FreqDist
import plotly.graph_objects as go
import plotly.express as px
lung_cancer=comment[comment['defind_cancer_with_nlp']=='มะเร็งปอด'].reset_index()['token']
sum_word = []
for i in range(len(lung_cancer)):
    sum_word+=lung_cancer[i]
    sum_word.remove('end')
sum_word
fdist = FreqDist(sum_word)
x=fdist.most_common(20)
word_count_table = pd.DataFrame(x,columns=['word','count'])

#วิธีการรักษา

In [ ]:
Treatment = ['ผ่าตัด','คีโม','ฉายเเสง','เเร่']
k = []
for i in range(len(comment)):
  L=[]
  for j in Treatment:
    if j in comment['token'][i]:
      L.append(j)
      k.append(L)
    else:
      k.append('ไม่สามารถระบุได้')
comment

In [ ]:
for i in range(len(comment)):
  print('ผ่าตัด' in comment['token'][i])

In [ ]:
k = pd.DataFrame()
l = []
for i in range(len(comment)):
  if 'แนะนำ' in comment['token'][i]:
    l.append(comment['token'][i])
k['word'] = l
k

12 = มะเร็งกล่องเสียง
19 = มะเร็งไทรอยด์
31 = มะเร็งโพรงจมูก
มะเร็งไขกระดูก


In [ ]:
fig = px.bar(word_count_table, x='count', y='word',text_auto=True,orientation='h')
fig.show()

In [ ]:
from sentence_transformers import SentenceTransformer,util
import numpy as np
sentences = list(comment['comments'])
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
embeddings = model.encode(sentences)
#Normalize the embeddings to unit length
Normalize_embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)
clustering_model = KMeans(n_clusters=4)
clustering_model.fit(embeddings)
cluster_assignment = clustering_model.labels_
print(cluster_assignment)
print(len(cluster_assignment))

In [ ]:
clusternd_sentences= {}
for sentence_id, cluster_id in enumerate(cluster_assignment):
  if cluster_id not in clusternd_sentences:
    clusternd_sentences[cluster_id] = []

  clusternd_sentences[cluster_id].append(sentences[sentence_id])

In [ ]:
clusternd_sentences

In [ ]:
# prompt: ทำการ concat Data_scraper_save (6) ถึง  Data_scraper_save (17)

import pandas as pd

# Assuming your files are named Data_scraper_save (6).csv, Data_scraper_save (7).csv, ... , Data_scraper_save (17).csv
# Replace with your actual file paths if needed

df_list = []
for i in range(6, 114):
  file_path = f'/content/Data_scraper_save ({i}).csv'  # Replace with your actual file path
  try:
    df = pd.read_csv(file_path, encoding='utf-8')
    df_list.append(df)
  except FileNotFoundError:
    print(f"File not found: {file_path}")

# Concatenate all DataFrames
final_df = pd.concat(df_list, ignore_index=True)
final_df_drop = final_df.drop_duplicates(subset=['comments'])
# Now you have a single DataFrame containing data from all files
print(final_df_drop)


In [ ]:
final_df_drop.to_csv('reddit_twothosan.csv',index=False)

In [ ]:
final_df_drop.groupby('name').sum().reset_index()

In [ ]:
# prompt: จากข้อมูล final_df_drop ช่วยสร้าง list เก็บชื่อที่ซ้ำ

duplicate_names = []
for name, group in final_df_drop.groupby('name'):
  if len(group) > 1:
    duplicate_names.append(name)


In [ ]:
final_df_drop[final_df_drop['name']=='myersmjsc']